In [4]:
import os.path as op
import os
# import sys
import numpy as np
import pandas as pd
import seaborn as sns
from fpdf import FPDF  

import mne
from mne.preprocessing import find_bad_channels_maxwell
import matplotlib.pyplot as plt
# from mne.time_frequency import psd_multitaper
from mne.preprocessing import annotate_muscle_zscore
from mne.preprocessing import ICA
from mne.preprocessing import read_ica
import scipy.stats

## Preprocessing

In [5]:
# data_path =  'D:\projects\WM_GRID\DATA\mg99a'
# cal_path = 'D:\projects\WM_GRID\DATA\ctc'
# sss_path = 'D:\projects\WM_GRID\DATA\sss'

rawdir =  'X:/pt_02884/actspace/rawdir/'
cal_path = 'X:/pt_02884/ctc'
sss_path = 'X:/pt_02884/sss'

subject_list = ['as01']

In [17]:
# %load 01-maxwell_filtering.py
"""
===================================
01. Maxwell filter using MNE-python
===================================
"""

def run_maxwell_filter(method = 'tsss'):

    crosstalk_file = op.join(cal_path, "ct_sparse_leipzig_061201.fif")
    fine_cal_file = op.join(sss_path, "sss_cal_3029-Leipzig_140903.dat")

    # Create empty dataframe for bad channel list
    df = pd.DataFrame()
    
    # Prepare PDF report
    pdf = FPDF(orientation="P", unit="mm", format="A4")
    
    print("Processing subject: %s" % subject_id)
    run = 0
    for file_name in file_names:
        run = run + 1
        print("  File: %s" % file_name)
        
        # Read raw data
        raw_fname_in = op.join(data_path, file_name + '.fif')
        raw = mne.io.read_raw_fif(
            raw_fname_in,
            allow_maxshield=True,
            preload=False,
            verbose=True)
        
        # Detect bad channels
        raw.info['bads'] = ['MEG0712','MEG1241','MEG2641']
        raw_check = raw.copy()
        auto_noisy_chs, auto_flat_chs, auto_scores = find_bad_channels_maxwell(
            raw_check, 
            cross_talk=crosstalk_file, 
            calibration=fine_cal_file,
            return_scores=True,
            verbose=True)
        raw.info['bads'].extend(auto_noisy_chs + auto_flat_chs)
        
        # Append bad channels to the list 
        df = df.append({'run': run,
                        'noisy': auto_noisy_chs, 
                        'flat': auto_flat_chs},
                        ignore_index=True)        
        
        # Visualize the scoring used to classify channels as noisy or flat
        ch_type = 'grad'
        fig = viz_badch_scores(auto_scores, ch_type)
        fname_fig = op.join(out_path,
                            "01_r%s_badchannels_%sscore.png" % (run,ch_type))
        fig.savefig(fname_fig)
        plt.close()
        ch_type = 'mag'
        fig = viz_badch_scores(auto_scores, ch_type)
        fname_fig = op.join(out_path,
                            "01_r%s_badchannels_%sscore.png" % (run,ch_type))
        fig.savefig(fname_fig)
        plt.close()
        
        # Fix Elekta magnetometer coil types
        raw.fix_mag_coil_types()
        # realign
        file_name_ref=file_names[0]
        raw_ref_fname_in = op.join(data_path, file_name_ref + '.fif')
        raw_ref = mne.io.read_raw_fif(raw_ref_fname_in,
            allow_maxshield=True,
            preload=False,
            verbose=True)
        dev_head_t_ref = raw_ref.info['dev_head_t']
        
        # Perform tSSS/SSS and Maxwell filtering
        raw_sss = mne.preprocessing.maxwell_filter(
            raw,
            origin='auto',
            cross_talk=crosstalk_file,
            calibration=fine_cal_file,
            st_duration=st_duration,
            coord_frame='head',
            destination=dev_head_t_ref,
            #coord_frame="meg", #only for empy room, comment it if using HPI
            verbose=True)
        
        # Show original and filtered signals
        fig = raw.copy().pick(['meg']).plot(duration=5,
                                            start=100,
                                            butterfly=True)        
        fname_fig = op.join(out_path,
                            '01_r%s_plotraw.png' % run)
        fig.savefig(fname_fig)
        plt.close()
        fig = raw_sss.copy().pick(['meg']).plot(duration=5,
                                                start=100,
                                                butterfly=True)
        fname_fig = op.join(out_path,
                            '01_r%s_plotraw%s.png' % (run,method))
        fig.savefig(fname_fig)
        plt.close()
        
        # Show original and filtered power
        fig1 = raw.plot_psd(picks = ['meg'],fmin = 1,fmax = 100)
        fname_fig1 = op.join(out_path,
                            '01_r%s_plot_psd_raw100.png' % run)
        fig1.savefig(fname_fig1)
        plt.close()
        fig2 = raw_sss.plot_psd(picks = ['meg'],fmin = 1,fmax = 100)
        fname_fig2 = op.join(out_path,
                            '01_r%s_plot_psd_raw100%s.png' % (run,method))
        fig2.savefig(fname_fig2)
        plt.close()
        
        # Save filtered data
        fname_out = op.join(out_path,
                            file_name + '_' + method + '.fif')
        raw_sss.save(fname_out, overwrite=True)
        
    # Save bad channel list
    df.to_csv(op.join(out_path,
                      '01_rAll_meg_badch_list.csv'),
              index=False)
    


def viz_badch_scores(auto_scores, ch_type):
    fig, ax = plt.subplots(1, 4, figsize=(12, 8))
    fig.suptitle(f'Automated noisy/flat channel detection: {ch_type}',
                  fontsize=16, fontweight='bold')
    
    #### Noisy channels ####
    ch_subset = auto_scores['ch_types'] == ch_type
    ch_names = auto_scores['ch_names'][ch_subset]
    scores = auto_scores['scores_noisy'][ch_subset]
    limits = auto_scores['limits_noisy'][ch_subset]
    bins = auto_scores['bins']  #the windows that were evaluated
    
    # Label each segment by its start and stop time (3 digits / 1 ms precision)
    bin_labels = [f'{start:3.3f} – {stop:3.3f}' 
                  for start, stop in bins]
    
    # Store  data in DataFrame
    data_to_plot = pd.DataFrame(data=scores,
                                columns=pd.Index(bin_labels, name='Time (s)'),
                                index=pd.Index(ch_names, name='Channel'))
    
    # First, plot the raw scores
    sns.heatmap(data=data_to_plot, 
                cmap='Reds', 
                cbar=False,
                # cbar_kws=dict(label='Score'),
                ax=ax[0])
    [ax[0].axvline(x, ls='dashed', lw=0.25, dashes=(25, 15), color='gray')
        for x in range(1, len(bins))]
    ax[0].set_title('Noisy: All Scores', fontweight='bold')

    # Second, highlight segments that exceeded the 'noisy' limit
    sns.heatmap(data=data_to_plot,
                vmin=np.nanmin(limits),
                cmap='Reds', 
                cbar=True, 
                # cbar_kws=dict(label='Score'), 
                ax=ax[1])
    [ax[1].axvline(x, ls='dashed', lw=0.25, dashes=(25, 15), color='gray')
        for x in range(1, len(bins))]
    ax[1].set_title('Noisy: Scores > Limit', fontweight='bold')
    
    #### Flat channels ####
    ch_subset = auto_scores['ch_types'] == ch_type
    ch_names = auto_scores['ch_names'][ch_subset]
    scores = auto_scores['scores_flat'][ch_subset]
    limits = auto_scores['limits_flat'][ch_subset]
    bins = auto_scores['bins']  #the windows that were evaluated
    
    # Label each segment by its start and stop time (3 digits / 1 ms precision)
    bin_labels = [f'{start:3.3f} – {stop:3.3f}' 
                  for start, stop in bins]
    
    # Store  data in DataFrame
    '''data_to_plot = pd.DataFrame(data=scores,
                                columns=pd.Index(bin_labels, name='Time (s)'),
                                index=pd.Index(ch_names, name='Channel'))'''
    
    # First, plot the raw scores
    sns.heatmap(data=data_to_plot, 
                cmap='Reds', 
                cbar=False,
                # cbar_kws=dict(label='Score'),
                ax=ax[2])
    [ax[2].axvline(x, ls='dashed', lw=0.25, dashes=(25, 15), color='gray')
        for x in range(1, len(bins))]
    ax[2].set_title('Flat: All Scores', fontweight='bold')

    # Second, highlight segments that exceeded the 'noisy' limit
    sns.heatmap(data=data_to_plot,
                vmax=np.nanmax(limits),
                cmap='Reds', 
                cbar=True,
                # cbar_kws=dict(label='Score'), 
                ax=ax[3])
    [ax[3].axvline(x, ls='dashed', lw=0.25, dashes=(25, 15), color='gray')
        for x in range(1, len(bins))]
    ax[3].set_title('Flat: Scores > Limit', fontweight='bold')
    
    # Fit figure title to not overlap with the subplots
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    return fig

In [ ]:
# %load 02 filter_downsample.py
def filter_downsample():
    
    print('\n\n\n\n\n#######################################################################################')
    print("Processing subject: %s" % subject_id)
    run = 0
    for file_name in file_names:
        run = run + 1
        print("  File: %s" % file_name)
        
        # Read raw data
        raw_fname_in = op.join(out_path,
                               file_name + '_sss.fif')
        raw = mne.io.read_raw_fif(
            raw_fname_in, 
            preload=True, 
            verbose='error')
        
        # Downsample copy of raw
        raw_resmpl = raw.copy()
        # Band-pass filter raw copy
        raw_resmpl.filter(1, 100)
        raw_resmpl.notch_filter([50])
        # Save filtered data
        fname_out = op.join(out_path,
                            file_name  + '_filter.fif')
        raw_resmpl.save(fname_out, overwrite=True)
        
    


In [ ]:
# %load 03-run_ica.py
"""
===========
03. Run ICA
===========
"""


def run_ica(max_iter = 100, n_components = 0.99, random_state = 1):
    
    print('\n\n\n\n\n#######################################################################################')
    print("Processing subject: %s" % subject_id)
    run = 0
    for file_name in file_names:
        run = run + 1
        print("  File: %s" % file_name)
        
        # Read raw data
        raw_fname_in = op.join(out_path,
                               file_name + '_filter.fif')
        raw = mne.io.read_raw_fif(
            raw_fname_in, 
            preload=True, 
            verbose='error')
        
        # Downsample copy of raw
        raw_resmpl = raw.copy().resample(sfreq)
            
        # Band-pass filter raw copy
        raw_resmpl.filter(l_freq, h_freq)
            
        # Concatenate raw copies
        if run == 1:
            raw_resmpl_all = mne.io.concatenate_raws([raw_resmpl])
        else:
            raw_resmpl_all = mne.io.concatenate_raws([raw_resmpl_all, raw_resmpl])
        
        del raw, raw_resmpl
    
    ###################
    # ICA on MEG data #
    ###################
    
    # Prepare PDF report
    pdf = FPDF(orientation="P", unit="mm", format="A4")
    
    # Define ICA settings
    ica = ICA(method=ica_method,
              random_state=random_state,
              n_components=n_components,
              verbose=True)
    
    # Run ICA on filtered raw data
    ica.fit(raw_resmpl_all,
            picks='meg',
            verbose=True)
    
    # Plot timecourse of estimated sources
    fig = ica.plot_sources(raw_resmpl_all,
                           start=100,
                           show_scrollbars=False,
                           title='ICA_MEG')
    
    

    fname_fig = op.join(out_path, 
                      "05_rAll_ica_meg_src.png")
    fig.savefig(fname_fig)
    plt.close(fig)
    

    # Project mixing matrix on interpolated sensor topography
    fig = ica.plot_components(title='ICA_MEG')
    for i in range(len(fig)):
        fname_fig = op.join(out_path, 
                            '05_rAll_ica_meg_cmp%d.png' % i)
        fig[i].savefig(fname_fig)
        plt.close(fig[i])
        
 
    # Save files
    ica_fname = op.join(out_path,
                        subject_id + 'ALL-ica_meg.fif')
    ica.save(ica_fname)
    


In [ ]:
# %load 04-extract_events.py
"""
===================
04. Extract events
===================

Extract events from the stimulus channel

"""


def run_events():
    
    # Prepare PDF report
    pdf = FPDF(orientation="P", unit="mm", format="A4")

    print("Processing subject: %s" % subject_id)
    run = 0
    for file_name in file_names:
        run = run + 1
        print("  File: %s" % file_name)
        
        # Read raw data
        run_fname = op.join(data_path,
                            file_name + '.fif')
        raw = mne.io.read_raw_fif(
            run_fname,
            allow_maxshield=True,
            verbose=True)
        
        ###############
        # Read events #
        ###############
    
        
        # Find all events
        events = mne.find_events(raw,
                                 stim_channel='STI101',
                                 consecutive = True,
                                 min_duration=0.001001,
                                 mask = 65280,
                                 mask_type = 'not_and'
                                )
        events = events[events[:,2] != 255]
        
        # Concatenate all events
        events = np.concatenate([events],axis = 0)
        events = events[events[:,0].argsort(),:]
        
        # Show events
        fig = mne.viz.plot_events(events)
        fname_fig = op.join(out_path,
                            "04_r%s_events.png" % run)
        fig.savefig(fname_fig)
        plt.close(fig)
        
 
        # Save event array
        fname_events = op.join(out_path,
                               file_name + '-eve.fif')                            
        mne.write_events(fname_events, events)
        


In [18]:
# %load config.py
"""
===========
Config file
===========

Configurate the parameters of the study.
"""
for subject_id in subject_list[:]:

    # =============================================================================
    # SESSION-SPECIFIC SETTINGS
    # =============================================================================

    # Set filename based on experiment number
    data_path = os.path.join(rawdir,subject_id+'a')
    file_exts = ['%sa1']

    file_names = [f % (subject_id) for f in file_exts]


    # =============================================================================
    # GENERAL SETTINGS
    # =============================================================================

    # Set out_path folder or create it if it doesn't exist
    out_path = op.join(data_path, "out_path")
    if not op.exists(out_path):
        os.mkdir(out_path)


    # =============================================================================
    # MAXWELL FILTERING SETTINGS
    # =============================================================================

    # Set filtering method
    method='sss'
    if method == 'tsss':
        st_duration = 10
    else:
        st_duration = None


    # =============================================================================
    # FILTERING AND DOWNSAMPLING SETTINGS
    # =============================================================================

    # Filter and resampling params
    l_freq = 1
    h_freq = 40
    sfreq = 200


    # =============================================================================
    # ICA SETTINGS
    # =============================================================================

    ica_method = 'fastica'
    n_components = 0.99
    max_iter = 800
    random_state = 1688


    # =============================================================================
    # RUN
    # =============================================================================

    run_maxwell_filter(method=method)

    #filter_downsample()

    #artifact_annotation()

    
    #run_ica(max_iter = max_iter, 
    #       n_components = n_components, 
    #        random_state = random_state)
    
    #run_events()

Processing subject: as01
  File: as01a1
Opening raw data file X:/pt_02884/actspace/rawdir/as01a\as01a1.fif...
    Read a total of 6 projection items:
        grad_ssp_upright2.fif : PCA-v1 (1 x 306)  idle
        grad_ssp_upright2.fif : PCA-v2 (1 x 306)  idle
        grad_ssp_upright2.fif : PCA-v3 (1 x 306)  idle
        mag_ssp_upright2.fif : PCA-v1 (1 x 306)  idle
        mag_ssp_upright2.fif : PCA-v2 (1 x 306)  idle
        mag_ssp_upright2.fif : PCA-v3 (1 x 306)  idle
    Range : 26000 ... 906999 =     26.000 ...   906.999 secs
Ready.
Applying low-pass filter with 40.0 Hz cutoff frequency ...
Reading 0 ... 880999  =      0.000 ...   880.999 secs...


C:\Users\peng\AppData\Local\Temp\3\ipykernel_8976\3627876290.py:27: RuntimeWarning: This filename (X:/pt_02884/actspace/rawdir/as01a\as01a1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 331 samples (0.331 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    2.5s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    6.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:   11.1s


Scanning for bad channels in 176 intervals (5.0 s) ...
102 of 102 magnetometer types replaced with T3.
    Bad MEG channels being reconstructed: ['MEG0712', 'MEG1241', 'MEG2641']
    Processing 204 gradiometers and 102 magnetometers
    Using fine calibration sss_cal_3029-Leipzig_140903.dat
        Adjusting non-orthogonal EX and EY
        Adjusted coil positions by (μ ± σ): 0.4° ± 0.4° (max: 1.9°)
    Automatic origin fit: head of radius 94.2 mm
    Using origin 1.7, 11.1, 44.3 mm in the head frame
        Interval   1:    0.000 -    4.999
        Interval   2:    5.000 -    9.999
        Interval   3:   10.000 -   14.999
        Interval   4:   15.000 -   19.999
        Interval   5:   20.000 -   24.999
        Interval   6:   25.000 -   29.999
        Interval   7:   30.000 -   34.999
        Interval   8:   35.000 -   39.999
        Interval   9:   40.000 -   44.999
        Interval  10:   45.000 -   49.999
        Interval  11:   50.000 -   54.999
        Interval  12:   55.000 -

AttributeError: 'DataFrame' object has no attribute 'append'

In [8]:
# %load config.py
"""
===========
Config file
===========

Configurate the parameters of the study.
"""
for subject_id in subject_list[:]:

    # =============================================================================
    # SESSION-SPECIFIC SETTINGS
    # =============================================================================

    # Set filename based on experiment number
    data_path = os.path.join(rawdir,subject_id+'a')
    file_exts = ['%s1']

    file_names = [f % (subject_id) for f in file_exts]


    # =============================================================================
    # GENERAL SETTINGS
    # =============================================================================

    # Set out_path folder or create it if it doesn't exist
    out_path = op.join(data_path, "out_path")
    if not op.exists(out_path):
        os.mkdir(out_path)


    # =============================================================================
    # MAXWELL FILTERING SETTINGS
    # =============================================================================

    # Set filtering method
    method='sss'
    if method == 'tsss':
        st_duration = 10
    else:
        st_duration = None


    # =============================================================================
    # FILTERING AND DOWNSAMPLING SETTINGS
    # =============================================================================

    # Filter and resampling params
    l_freq = 1
    h_freq = 40
    sfreq = 200


    # =============================================================================
    # ICA SETTINGS
    # =============================================================================

    ica_method = 'fastica'
    n_components = 0.99
    max_iter = 800
    random_state = 1688


    # =============================================================================
    # RUN
    # =============================================================================

    run_maxwell_filter(method=method)

    #filter_downsample()

    #artifact_annotation()

    
    #run_ica(max_iter = max_iter, 
    #       n_components = n_components, 
    #        random_state = random_state)
    
    #run_events()

Processing subject: as01
  File: as011
Opening raw data file X:/pt_02884/actspace/rawdir/as01a\as011.fif...


C:\Users\peng\AppData\Local\Temp\3\ipykernel_8976\2005254866.py:27: RuntimeWarning: This filename (X:/pt_02884/actspace/rawdir/as01a\as011.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(


FileNotFoundError: fname does not exist: "X:\pt_02884\actspace\rawdir\as01a\as011.fif"

In [ ]:
# %load 05-apply_ica.py
"""
===============
05. Apply ICA
===============
"""

def apply_ica(meg_ica_eog = [], meg_ica_ecg = []):

    # Prepare PDF report
    pdf = FPDF(orientation="P", unit="mm", format="A4")
    
    print("Processing subject: %s" % subject_id)
    run = 0
    for file_name in file_names:
        run = run + 1
        print("  File: %s" % file_name)
        
        # Read raw data
        raw_fname_in = op.join(out_path,
                               file_name + '_filter.fif')
        raw = mne.io.read_raw_fif(
            raw_fname_in, 
            preload=True, 
            verbose='error')
        
        # Show original signal

        chs = ['MEG0311', 'MEG0121', 'MEG1211', 'MEG1411']
        chan_idxs = [raw.ch_names.index(ch) for ch in chs]
        fig1 = raw.plot(order=chan_idxs,
                       duration=50,
                       start=100)        
        fname_fig1 = op.join(out_path,
                            '06_r%s_ica_raw0.png' % run)
        fig1.savefig(fname_fig1)
        plt.close()
        
        ###################
        # ICA on MEG data #
        ###################
        
        if [meg_ica_eog + meg_ica_ecg] != []:
            
            # Restore ICA solution from fif file
            ica_meg_fname = op.join(out_path,
                               subject_id + 'ALL-ica_meg.fif')
            ica_meg = read_ica(ica_meg_fname)
            
            # Select EOG- and ECG-related components for exclusion
            ica_meg.exclude.extend(meg_ica_eog + meg_ica_ecg)
            
        
        ###################
        
        # Remove selected components from the signal  
        raw_ica = raw.copy()
        ica_meg.apply(raw_ica)
        
        # Show cleaned signal
        fig_ica = raw_ica.plot(order=chan_idxs,
                               duration=50,
                               start=100)        
        fname_fig_ica = op.join(out_path,
                                '06_r%s_ica_rawICA.png' % run)
        fig_ica.savefig(fname_fig_ica)
        plt.close()
        
        '''
        # Add figures to report
        pdf.ln(120)
        pdf.cell(0, 10, 'Timecourse of output data', 'B', ln=1)
        pdf.image(fname_fig_ica, 0, 175, pdf.epw)
        '''
        # Save cleaned raw data
        fname_out = op.join(out_path,
                            file_name + '_ica.fif')
        raw_ica.save(fname_out,overwrite=True)
    




In [ ]:
# %load 06-make_epochs.py
"""
====================
06. Make epochs
====================
"""


def run_epochs():
    
    # Prepare PDF report
    pdf = FPDF(orientation="P", unit="mm", format="A4")
    
    print("Processing subject: %s" % subject_id)
    
    # Create empty lists
    raw_list = list()
    events_list = list()
    # metadata_list = list()
    
    print("Processing subject: %s" % subject_id)
    run = 0
    for file_name in file_names:
        run = run + 1
        print("  File: %s" % file_name)
        
        # Read raw data
        raw_fname_in = op.join(out_path,
                               file_name + '_ica.fif')
        raw_tmp = mne.io.read_raw_fif(
            raw_fname_in, 
            preload=True, 
            verbose='error')
        
        # Read events
        events_tmp = mne.read_events(op.join(out_path,
                                             file_name + '-eve.fif'))                           
        
        # Append read data to list
        raw_list.append(raw_tmp)
        events_list.append(events_tmp)
        # metadata_list.append(metadata_tmp)

    # Concatenate raw instances as if they were continuous
    raw, events = mne.concatenate_raws(raw_list,
                                       events_list=events_list)
    del raw_list
    
    
    # Epoch raw data
    epochs = mne.Epochs(raw,
                        events, 
                        events_id,
                        tmin, tmax,
                        baseline=None,
                        proj=True,
                        picks='all',
                        detrend=1,
                        reject_by_annotation=False,
                        verbose=True)
    
    # epochs.metadata = metadata
    
    del raw
    
    # Plot percentage of rejected epochs per channel
    fig1 = epochs.plot_drop_log()
    fname_fig1 = op.join(out_path,
                        '07_rAll_epoch_drop.png')
    fig1.savefig(fname_fig1)
    plt.close()
    
    # Plot evoked by epoch
    fig2 = epochs.plot(picks='meg',
                      title='meg',
                      n_epochs=10)
    fname_fig2 = op.join(out_path,
                        '07_rAll_epoch_evk.png')
    fig2.savefig(fname_fig2)
    plt.close(fig2)
    

    # Count the number of epochs defined by different events
    # Save epoched data
    epochs.save(op.join(out_path,
                        file_names[0][0:13] + 'ALL_epo.fif'),                           
                    overwrite=True)





In [ ]:
#mg01
meg_ica_eog = [11]
meg_ica_ecg = [7]


In [ ]:
# %load config.py
"""
===========
Config file
===========

Configurate the parameters of the study.
"""

for subject_id in subject_list[:1]:

    # =============================================================================
    # SESSION-SPECIFIC SETTINGS
    # =============================================================================
    # Set filename based on experiment number
    data_path = op.join(rawdir,subject_id+'a')
    file_exts = ['%sa01',
                     '%sa02',
                     '%sa03',
                     '%sa04',
                     '%sa05',
                     '%sa06',
                     '%sb07',
                     '%sb08',
                     '%sb09',
                     '%sb10',
                     '%sb11',
                     '%sb12']#,

    file_names = [f % (subject_id) for f in file_exts]


    # =============================================================================
    # GENERAL SETTINGS
    # =============================================================================

    # Set out_path folder or create it if it doesn't exist
    out_path = op.join(data_path, "out_path")
    if not op.exists(out_path):
        os.mkdir(out_path)
    noreject_path = op.join(out_path,"noreject_path")    
    if not op.exists(noreject_path):
        os.mkdir(noreject_path)


    # =============================================================================
    # MAXWELL FILTERING SETTINGS
    # =============================================================================

    # Set filtering method
    method='sss'
    if method == 'tsss':
        st_duration = 10
    else:
        st_duration = None


    # =============================================================================
    # FILTERING AND DOWNSAMPLING SETTINGS
    # =============================================================================

    # Filter and resampling params
    l_freq = 1
    h_freq = 40
    sfreq = 200


    # =============================================================================
    # EPOCHING SETTINGS
    # =============================================================================

    # Set timewindow
    tmin = -0.5
    tmax = 8.1

    # Epoch rejection criteria
    reject_meg = dict(grad=4000e-13,    # T / m (gradiometers)
                      mag=4e-12         # T (magnetometers)
                      )

    # Set epoching event ids

    events_id = 101



    # =============================================================================
    # ICA SETTINGS
    # =============================================================================

    ica_method = 'fastica'
    n_components = 0.99
    max_iter = 800
    random_state = 1688


    # =============================================================================
    #  FACTOR AND CONDITIONS OF INTEREST
    # =============================================================================

    
    # factor = 'Category'
    # conditions = ['face', 'object', 'letter', 'false']

        # factor = 'Relevance'
        # conditions = ['Relevant target','Relevant non-target','Irrelevant']


    # =============================================================================
    # TIME-FREQUENCY REPRESENTATION SETTINGS
    # =============================================================================

    baseline_w = [-0.5, -0.25]     #only for plotting
    freq_band = 'both' #can be 'low', 'high' or 'both'

    # =============================================================================
    # RUN
    # =============================================================================
    
    apply_ica(meg_ica_eog = meg_ica_eog,
            meg_ica_ecg = meg_ica_ecg)

    run_epochs()
